# Bar Crawl Exercise


# Setup

Import packages and load data. I used {get reference] to convert the original .csv files into .ftr (feather) to reduce size and load times. 

In [ ]:
import opendatasets as od
from BarCrawl_helpers import *

import numpy as np
import pandas as pd

from scipy.signal import spectrogram
from scipy.interpolate import interp1d

from sklearn.decomposition import FastICA, PCA

import matplotlib.pyplot as plt

from os import listdir, walk, environ
from os.path import isfile, join, exists

import pickle

import re as re
from tqdm import tqdm

import sys
import warnings



warnings.filterwarnings('ignore')

In [22]:
TAC_limit = 0.08

# Cartesian Accelerations and their ICA signals
xyz = ['x', 'y', 'z']
#C = ['C1', 'C2', 'C3']
C = ['C1', 'C2']

# Spherical Accelerations and their ICA signals
rtp = ['rho', 'theta', 'phi']
#S = ['S1', 'S2', 'S3']
S = ['S1', 'S2']

# Fetch Data and Initial Inspection

 - get data from Kaggle
 - create data folder
 - reduce memory usage (see citation for borrowed code snippet)
 - convert .csv to .ftr (feather) for faster loading in future runs


In [ ]:
## Fetch Data

In [ ]:
if not exists('./feather_data'):
    od.download("https://www.kaggle.com/datasets/nautiyalamit/bar-crawl-detecting-heavy-drinking-data-set")

In [ ]:
# The code in this cell is credited to 
# https://gist.github.com/fujiyuu75/748bc168c9ca8a49f86e144a08849893

from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype

def reduce_mem_usage(df, use_float16=False):
    """
    Iterate through all the columns of a dataframe and modify the data type to reduce memory usage.        
    """
    
    start_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
    
    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
# kaggle datasets download -d nautiyalamit/bar-crawl-detecting-heavy-drinking-data-set

## Make Dataframes and Split Accelerometer Data by Subject 

In [ ]:
datafolder = 'feather_data'
acc_datafile = 'all_accelerometer_data_pids_13.ftr'
phone_datafile = 'phone_types.ftr'

all_acc_df = pd.read_feather(join(datafolder, acc_datafile))
phones_df = pd.read_feather(join(datafolder, phone_datafile))

In [ ]:
all_acc_df.head()

In [ ]:
# make a dictionary of TAC files with keys being subject PIDs
# # pattern = re.compile(r'\D{2}\d{4}')
# pattern = re.compile(r'\D{2}\d{4}')
# tac_files = [filename if re.match(r'\D{2}\d{4}',filename) for filename in listdir(datafolder)].sort()

tac_files = sorted([filename for filename in listdir(datafolder) if re.match(r'\D{2}\d{4}',filename)])
pid_list = [filename[0:6] for filename in tac_files]

tac_df = {}

for tac_file in tac_files:
    pid = tac_file[0:6]
    if tac_file.endswith('.ftr'):
        tac_df[pid] = pd.read_feather(join(datafolder, tac_file))
    elif tac_file.endswith('.csv'):
        tac_df[pid] = pd.read_csv(join(datafolder, tac_file))
    else:
        print('%s is neither a .csv or .ftr' % tac_file)

In [ ]:
acc_df = {}

for pid in pid_list:
    acc_df[pid] = all_acc_df[all_acc_df.pid==pid].reset_index().drop(columns = 'index')

## Initial inspection

Notes:

 - acceleration time stamps in milliseconds sampled nominally at 40 Hz
 - some subjects' acceleration data starts with a spurious zero reading
 - TAC time stamps in seconds

In [ ]:
for pid in pid_list:
    print(acc_df[pid].head(2))

In [ ]:
for pid in pid_list:
    print(tac_df[pid].head(2))

In [ ]:
fig, ax = plt.subplots(3,5, figsize=(12,7), sharex=True, sharey = True)

direction = 'x';

for k, pid in enumerate(pid_list):
    acc_df[pid].iloc[1:-1].plot(x='time', y = direction, marker = '.', markersize = 1, linestyle = 'none', ax = ax[k%3, int(np.floor(k/3))], legend=False)
    ax[k%3, int(np.floor(k/3))].set_title(pid)
    
fig.suptitle(f'{direction}-Acceleration Data')
fig.delaxes(ax[1,4])
fig.delaxes(ax[2,4])
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(3,5, figsize=(12,7), sharey=True)


for k, pid in enumerate(pid_list):
    tac_df[pid].iloc[1:-1].plot(x='timestamp', y = 'TAC_Reading', marker = '.', markersize=1, linestyle = 'none', ax = ax[k%3, int(np.floor(k/3))], legend=False)
    ax[k%3, int(np.floor(k/3))].set_title(pid)
    
fig.suptitle('TAC Data')
fig.delaxes(ax[1,4])
fig.delaxes(ax[2,4])
plt.tight_layout()
plt.show()

## Observations

 - every subject has patches of missing data
 - subjects JB3156 and CC6740 accelerometer data looks categorically different; coincidentally, both subjects use Android phones. For now, I'll restrict the analysis to iPhone data.
 - TAC data looks okay

# Clean Up and Transform Data

 - Remove unusable subject data
 - Convert accelerometer time to seconds and resample TAC to align with accelerometer timestamps
 - Snip out 8 sec clips. Why 8 sec? We're attempting to infer blood alcohol levels from accelerations induced by the subjects' movement patterns. If the acceleration data encodes bouts of walking, a rhythmic behavior, the frequency content of acceleration signals could be useful. Typically, h Most notably,  though we may also find accelerations suggesting the subject has fallen or dropped their phone.


In [ ]:
pid_remove = ['JB3156', 'CC6740']

for pid in pid_remove:
    if pid in pid_list:
        pid_list.remove(pid)
    acc_df.pop(pid, None)
    tac_df.pop(pid, None)

In [ ]:
for pid in pid_list:
    try:
        tac_df[pid].rename(columns={'timestamp':'time_s'}, inplace=True)
        acc_df[pid]['time_s'] = acc_df[pid]['time'].to_numpy()/1000
        del acc_df[pid]['time']
    except:
        print('already been done')
    
    linear_interp = interp1d(tac_df[pid]['time_s'].to_numpy(), tac_df[pid]['TAC_Reading'], kind='linear', axis=0, bounds_error=False)
    acc_df[pid]['tac'] = linear_interp(acc_df[pid]['time_s'].to_numpy())
    
    acc_df[pid].dropna(axis=0, how = 'any', inplace=True)   # tac readings extend past accelerometer readings, so trim down where extrapolation gives Null tac values
    
    tac_df[pid]['time_s'] = tac_df[pid]['time_s'].to_numpy() - acc_df[pid]['time_s'][0]
    acc_df[pid]['time_s'] = acc_df[pid]['time_s'].to_numpy() - acc_df[pid]['time_s'][0]


In [ ]:
fig, ax = plt.subplots(3,4, figsize=(20, 10), sharex=True, sharey=True)

for k, pid in enumerate(pid_list):
    tac_df[pid].plot(x='time_s', y = 'TAC_Reading', ax = ax[k%3, int(np.floor(k/3))], \
                                     legend=False)
    acc_df[pid].plot(x='time_s', y = 'tac', ax = ax[k%3, int(np.floor(k/3))], \
                                     marker = '.', linestyle = 'none',\
                                     legend=False)
    ax[k%3, int(np.floor(k/3))].plot(acc_df[pid].time_s, np.zeros(len(acc_df[pid].time_s)), \
                                                    marker = '.', linestyle = 'none')

    ax[k%3, int(np.floor(k/3))].set_title(pid, pad = -10)

lines = ax[0,0].get_lines()
labels = ['original TAC', 'interpolated TAC', 'Support']
fig.legend(lines, labels)

fig.delaxes(ax[2,3])
plt.tight_layout()
plt.show()

Explain windowing. overlap = 0.5

Transforming signals

Glossary:

pid
sid
x,y,z
rho, theta, phi

fftx,ffty,fftz
fftr,fftt,fftp

In [ ]:
# Cartesian Accelerations and their ICA signals
xyz = ['x', 'y', 'z']
#C = ['C1', 'C2', 'C3']
C = ['C1', 'C2']

# Spherical Accelerations and their ICA signals
rtp = ['rho', 'theta', 'phi']
#S = ['S1', 'S2', 'S3']
S = ['S1', 'S2']

#ica = FastICA(n_components=2, algorithm = 'parallel', whiten = 'arbitrary-variance', tol=1e-3, max_iter=500)
pca = PCA(n_components=2)

In [ ]:
win_sec = 10  # sliding window in seconds
win = win_sec * 40   # sliding window size in number of samples

win_14 = int(win/4)
win_12 = int(win/2)
win_34 = int(3*win/4)

win_slide = win_34


subject_data = {}

for pid in pid_list:
    print(f"{pid}...")
    
    acc_df[pid].dropna(axis=0, how = 'any')
    gap_idx = np.argwhere([np.diff(acc_df[pid].time_s)>0.25])[:,1]
    gap_idx = np.append(np.array([0]), gap_idx)

    gap_pairs = np.vstack((gap_idx[0:-2], gap_idx[1:-1])).transpose()
    gap_sizes = np.squeeze(np.diff(gap_pairs, axis=1))
    
    sample = []
    sample_fft = []
    sample_count = 0
    
    for k, gap in tqdm(enumerate(gap_sizes)):
        if gap>win:
            start_idx = int(((gap%(win_34))/2) + gap_idx[k])
            while (start_idx + win) < gap_pairs[k, 1]:
                s = acc_df[pid].iloc[start_idx:(start_idx+win)].reset_index().drop(columns='index')
                s = s.assign(sid=sample_count)
                
                s[rtp] = cart_to_spherical(s[xyz].to_numpy())
                # mmm = np.linalg.norm(s[xyz].to_numpy(), axis = 1)
                # s['rho'] = np.linalg.norm(s[xyz].to_numpy(), axis = 1)   # Just using rho, vector magnitude
                
                s[C] = pca.fit_transform(s[xyz].to_numpy())
                s[S] = pca.fit_transform(s[rtp].to_numpy())
                
                sf = s[['pid', 'sid']].iloc[0:win_14]
                
                sf['freq'] = np.fft.fftfreq(win, 0.025)[0:win_14]
                sf[xyz] = np.abs(np.fft.fft(s[xyz].to_numpy(), axis=0))[0:win_14,:]
                sf[rtp] = np.abs(np.fft.fft(s[rtp].to_numpy(), axis=0))[0:win_14,:]
                
                sf[C] = np.abs(np.fft.fft(s[C].to_numpy(), axis=0))[0:win_14,:]
                sf[S] = np.abs(np.fft.fft(s[S].to_numpy(), axis=0))[0:win_14,:]
                
                sample_count+=1
                
                sample.append(s[['pid', 'sid', 'tac', 'time_s'] + xyz + C + rtp + S])
                sample_fft.append(sf[['pid', 'sid', 'freq'] + xyz + C + rtp + S])
                
                start_idx += win_slide
    
    sample_df = pd.concat(sample).reset_index()
    sample_df.drop(columns = 'index', inplace=True)
    
    sample_fft_df = pd.concat(sample_fft).reset_index()
    sample_fft_df.drop(columns = 'index', inplace=True)

    subject_data[pid] = {'sample':sample_df, 'sample_fft':sample_fft_df}

print('...done.')

In [ ]:
pickle.dump(subject_data, open("subject_data.p","wb"))

# Statistics, normalization, and feature creation

In [1]:
import opendatasets as od
from BarCrawl_helpers import *

import numpy as np
import pandas as pd

from scipy.signal import spectrogram
from scipy.interpolate import interp1d

from sklearn.decomposition import FastICA, PCA

import matplotlib.pyplot as plt

from os import listdir, walk, environ
from os.path import isfile, join, exists

import pickle

import re as re
from tqdm import tqdm

import sys
import warnings



warnings.filterwarnings('ignore')

In [4]:
if exists('subject_data.p'):
    subject_data =pickle.load(open("subject_data.p","rb"))
    
pid_list = list(subject_data.keys())

pp = pid_list[0]

In [5]:
subject_data[pp]['sample'].head()

,pid,sid,tac,time_s,x,y,z,C1,C2,rho,theta,phi,S1,S2
0,BK7610,0,0.056953,2.451,-0.004002,0.0052,0.0010,-0.007883,0.003459,0.006637,2.226662,1.419537,53.152622,0.029924
1,BK7610,0,0.056954,2.476,-0.000500,0.0302,0.0227,0.002352,-0.028041,0.037783,1.587354,0.926298,52.513302,-0.463420
2,BK7610,0,0.056954,2.500,-0.009102,0.0300,0.0420,0.021428,-0.033232,0.052410,1.865361,0.641213,52.791313,-0.748561
3,BK7610,0,0.056954,2.527,-0.018997,-0.0031,0.0049,0.000973,0.013144,0.019862,3.303344,1.321519,54.229298,-0.068150
4,BK7610,0,0.056954,2.551,-0.016693,-0.0209,-0.0183,-0.014145,0.037377,0.032409,4.038433,2.170805,54.964397,0.781079


In [6]:
subject_data[pp]['sample_fft'].head()
# stats_fft_temp = subject_data[pid]['sample_fft'].drop(columns='freq').groupby(['pid', 'sid']).agg([power, dc, peak, peak_freq])

,pid,sid,freq,x,y,z,C1,C2,rho,theta,phi,S1,S2
0,BK7610,0,0.0,0.378693,1.490600,3.918900,3.325113e-08,2.854231e-07,22.043338,20370.383392,555.755966,0.000844,0.000033
1,BK7610,0,0.1,0.800844,0.361167,1.319130,1.319580e+00,2.885166e-01,2.160941,5761.442678,14.831388,5761.442603,14.847070
2,BK7610,0,0.2,1.028118,0.194124,1.900259,1.723881e+00,8.110972e-01,7.182344,2793.766444,6.572983,2793.767203,6.613408
3,BK7610,0,0.3,1.771755,0.063159,1.914804,1.951763e+00,5.754892e-01,2.679532,2055.167258,24.323244,2055.167431,24.309304
4,BK7610,0,0.4,1.190414,0.189216,2.313085,2.183559e+00,9.284566e-01,3.242622,1081.244810,19.618106,1081.244380,19.614105


In [14]:
subject_stats = {}

for pid in tqdm(pid_list):
    print(pid)
    
    stats_temp = subject_data[pid]['sample'].drop(columns = 'time_s').groupby(['sid']).agg(['mean', 'std', 'max', 'min'])
    stats_temp.drop(columns = ['tac_min', 'tac_mean', 'tac_std'], inplace=True, errors = 'ignore')
    stats_temp.rename(columns = {'tac_max':'tac'}, inplace=True)
    
    # stats_fft_temp = subject_data[pid]['sample_fft'].drop(columns='freq').groupby(['pid', 'sid']).agg([power, dc, peak, peak_freq])
    stats_fft_temp = subject_data[pid]['sample_fft'].drop(columns='freq').groupby(['sid']).agg([power, peak, peak_freq])

    
    # join time-domain and frequency-domain stats
    subject_stats[pid] = stats_temp.join(stats_fft_temp)
    
    
    # combine the multi-index column names into single string
    col_names = subject_stats[pid].columns.to_flat_index().values
    subject_stats[pid].columns = ['_'.join(col_name) for col_name in col_names]
    
    subject_stats[pid].drop(columns = ['tac_min', 'tac_mean', 'tac_std'], inplace=True, errors = 'ignore')
    subject_stats[pid].rename(columns = {'tac_max':'tac'}, inplace=True)
    
print('...done.')

  0%|          | 0/11 [00:00<?, ?it/s]

BK7610


  9%|▉         | 1/11 [00:10<01:44, 10.46s/it]

BU4707


 18%|█▊        | 2/11 [00:14<01:02,  6.91s/it]

...

SA0297


 91%|█████████ | 10/11 [00:45<00:04,  4.06s/it]

SF3079


100%|██████████| 11/11 [00:53<00:00,  4.82s/it]

...done.


In [19]:
# Check that the data are unique across subjects and within subjects

# for pid in pid_list:
#     print(f'\n{pid}')
#     display(subject_stats[pid].head(2))
#     display(subject_stats[pid].tail(2))

## Summary statistics, normalization, and feature creation

In [89]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [61]:
targets = {}
features = {}

for pid in pid_list:
    targets[pid] = pd.DataFrame({'tac':subject_stats[pid]['tac'],
                                'isOverLimit': 1.0*(subject_stats[pid]['tac']>TAC_limit)})
    
    features[pid] = subject_stats[pid].drop(columns='tac')

In [62]:
scaler = StandardScaler()
cols = features[pid].columns

for pid in pid_list:
    data_scaled = scaler.fit_transform(features[pid])
    features[pid] = pd.DataFrame(data_scaled, columns = cols)

In [63]:
features_df = pd.concat(features)
targets_df = pd.concat(targets)

In [68]:
features_df.columns

Index(['x_mean', 'x_std', 'x_max', 'x_min', 'y_mean', 'y_std', 'y_max',
       'y_min', 'z_mean', 'z_std', 'z_max', 'z_min', 'C1_mean', 'C1_std',
       'C1_max', 'C1_min', 'C2_mean', 'C2_std', 'C2_max', 'C2_min', 'rho_mean',
       'rho_std', 'rho_max', 'rho_min', 'theta_mean', 'theta_std', 'theta_max',
       'theta_min', 'phi_mean', 'phi_std', 'phi_max', 'phi_min', 'S1_mean',
       'S1_std', 'S1_max', 'S1_min', 'S2_mean', 'S2_std', 'S2_max', 'S2_min',
       'x_power', 'x_peak', 'x_peak_freq', 'y_power', 'y_peak', 'y_peak_freq',
       'z_power', 'z_peak', 'z_peak_freq', 'C1_power', 'C1_peak',
       'C1_peak_freq', 'C2_power', 'C2_peak', 'C2_peak_freq', 'rho_power',
       'rho_peak', 'rho_peak_freq', 'theta_power', 'theta_peak',
       'theta_peak_freq', 'phi_power', 'phi_peak', 'phi_peak_freq', 'S1_power',
       'S1_peak', 'S1_peak_freq', 'S2_power', 'S2_peak', 'S2_peak_freq'],
      dtype='object')

In [87]:
X = features_df.to_numpy()
y = targets_df['isOverLimit'].to_numpy()

In [155]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

frac = lambda x: sum(x)/len(x)
test_frac = frac(y_train)
train_frac = frac(y_test)
total = sum(y_train) + sum(y_test)

print(f"""Fraction of training and testing sets which are positives: 
train:\t{train_frac}
test:\t{test_frac}

Total positives: {total}
(this number should not change regardless of split)""")

Fraction of training and testing sets which are positives: 
train:	0.47103274559193953
test:	0.47366763756592933

Total positives: 7513.0
(this number should not change regardless of split)


In [149]:
rafo_classifier = RandomForestClassifier(max_depth)

In [150]:
rafo_classifier.fit(X_train, y_train)

RandomForestClassifier()

In [151]:
y_pred = rafo_classifier.predict(X_test)

In [159]:
metrics.f1_score(y_test, y_pred)
fi = rafo_classifier.feature_importances_

idx = np.argsort(-1*fi)

for i in idx:
    print(f'Feature {cols[i]}: {fi[i]}')

Feature S1_peak_freq: 0.041705247215294824
Feature theta_peak_freq: 0.03721323329047127
Feature x_mean: 0.03250156032865696
Feature phi_max: 0.02970009416823723
Feature phi_power: 0.029001270923525946
Feature phi_mean: 0.026375703188837186
Feature x_peak: 0.02591388991425359
Feature x_max: 0.023874923885652544
Feature S2_max: 0.020331058278309406
Feature y_mean: 0.020298786692967852
Feature y_power: 0.01974822516012416
Feature x_min: 0.01915634538491681
Feature y_peak: 0.018242032978517446
Feature z_mean: 0.01808335290154686
Feature rho_peak_freq: 0.017801139583967184
Feature rho_peak: 0.01716767804216254
Feature y_min: 0.01672686386145732
Feature x_std: 0.015589413467628514
Feature rho_mean: 0.01544106061236458
Feature C2_peak: 0.015421944209995328
Feature y_std: 0.015408639538966708
Feature S2_min: 0.015122425841829611
Feature C1_power: 0.01496415756396607
Feature z_min: 0.01494503018791572
Feature C2_min: 0.014429095284258273
Feature rho_min: 0.013973685116194656
Feature S2_peak_fre

In [114]:
# PCcols = [c for c in features_df.columns if (c.startswith('S') or c.startswith('C'))]

PCcols = features_df.columns[:]

Xc = features_df[PCcols].to_numpy()
Xc_train, Xc_test, y_train, y_test = train_test_split(Xc, y, test_size=0.2)

In [115]:
rafo_classifier.fit(Xc_train, y_train)
yc_pred = rafo_classifier.predict(Xc_test)

In [116]:
metrics.f1_score(y_test, yc_pred)

0.8407470288624789

In [ ]:
stats = []
stats_fft = []

for s,sf in zip(sample, sample_fft):
    s_temp = s.drop(columns = 'time_s').groupby(['pid', 'sid']).agg(['mean', 'std', 'max', 'min'])
    s_temp.drop(columns = ['tac_min', 'tac_mean', 'tac_std'], inplace=True, errors = 'ignore')
    s_temp.rename(columns = {'tac_max':'tac'}, inplace=True)
    
    stats.append(s_temp)
    
    sf_temp = sf.drop(columns='freq').groupby(['pid', 'sid']).agg([power, dc, peak, peak_freq])
    stats_fft.append(sf_temp)

In [ ]:
sample_df.head()

In [ ]:
null_count(sample_fft_df)

In [ ]:
stats_df = sample_df[sample_df['pid']==pid].groupby(['pid', 'sid']).agg(['mean', 'std', 'max','min'])

In [ ]:
stats_df.plot(x = ('time_s', 'mean'), y=('tac','mean'), marker = '.')

In [ ]:
len(sample_df)/len(stats_df)

In [ ]:
stats_df.columns = ['_'.join(col_names) for col_names in stats_df.columns.to_flat_index().values]

In [ ]:
stats_df.head()

In [ ]:
stats_df.drop(columns = ['tac_min', 'tac_mean', 'tac_std'], inplace=True, errors = 'ignore')
stats_df.rename(columns = {'tac_max':'tac'}, inplace=True)
stats_df['tac_over_limit'] = stats_df['tac'].to_numpy()>=0.08
stats_df.head()

In [ ]:
stats_fft_df = sample_fft_df[sample_fft_df['pid']=='BK7610'].drop(columns='freq').groupby(['pid', 'sid']).agg([power, dc, peak, peak_freq])
stats_fft_df.head()

In [ ]:
stats_fft_df.columns = ['_'.join(col_names) for col_names in stats_fft_df.columns.to_flat_index().values]

In [ ]:
stats_fft_df.head()

In [ ]:
features_df = stats_df.join(stats_fft_df, on = ['pid','sid'])
features_df.columns

In [ ]:
features_df['tac'].hist(bins=100)

In [ ]:
null_count(features_df)

In [ ]:
features_df.groupby('tac_over_limit')['tac'].count()

# Modeling, Validation, and Model Selection

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
targets_df = features_df.tac

In [ ]:
for col in features_df.columns:
    print(f"{col}: {features_df[col].isna().sum()}")

## Summary

# Sandbox below

In [ ]:
np.abs(sample[1].fftx.to_numpy())[0:50]

In [ ]:
# fftx = 
# # for s in tqdm(sample):
# #     fftx
#sample_fft[1200].plot(x = 'freq', y = ['x', 'y', 'z', 'C1', 'C2'])
sample_fft_df[sample_fft_df['pid'] == 'BK7610'].plot(x = 'freq', y = 'C2', marker = '.', linestyle= 'none', alpha = 0.1)

plt.xlim([-0.1, 5])

In [ ]:
# win_sec = 10  # sliding window in seconds
# win = win_sec * 40   # sliding window size in number of samples
# win_34 = int(3*win/4)

# sample_df = pd.DataFrame()
# for pid in pid_list:
#     gap_idx = np.argwhere([np.diff(acc_df[pid].time_s)>0.25])[:,1]
#     gap_idx = np.append(np.array([0]), gap_idx)

#     gap_pairs = np.vstack((gap_idx[0:-2], gap_idx[1:-1])).transpose()
#     gap_sizes = np.squeeze(np.diff(gap_pairs, axis=1))
#     for k, gap in enumerate(gap_sizes):
#         if gap>win:
#             start_idx = int(((gap%(win_34))/2) + gap_idx[k])
#             while (start_idx + win) < gap_pairs[k, 1]:
#                 s = acc_df[pid].iloc[start_idx:(start_idx+win)].reset_index().drop(columns='index')
#                 s = s.assign(sid=k)
#                 sample_df = pd.concat([sample_df, s])
#                 start_idx += win_34  

In [ ]:
test_df = sample[1].copy()
# test_df[['rho', 'theta', 'phi']] = test_df[['x', 'y', 'z']].apply(cart_to_spherical, axis=1, result_type='expand')
# test_df[['x', 'y', 'z']].apply(cart_to_spherical, axis=1, result_type='expand')

test_df[['rho', 'theta', 'phi']] = cart_to_spherical(test_df[['x', 'y', 'z']].to_numpy())

# r, t, p = cart_to_spherical(xyz)
test_df

In [ ]:
sample[1][['x','y','z']].agg(['mean', 'std', q25, q50, q75])

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (20, 2))
plt.plot(acc_df[pid].time, np.ones(np.size(acc_df[pid].time)), '.')
plt.plot(acc_df[pid]['time'].iloc[gap_idx], np.ones(len(gap_idx)), '.')

#plt.plot(np.diff(acc_df[pid].time))
print(pid)

In [ ]:
type(sample)

In [ ]:
stats = ['mean', 'std', 'max', q75, q50, q25, 'min']
sample_grouped = sample[1][['x','y','z']].agg(stats).stack().to_frame().transpose()

sample_grouped

In [ ]:
data = []
for k in range(5):
    data.append([np.random.random((1,5))])
    
AA = pd.DataFrame(data, columns=['b'])


AA.head()
# for k in range(10):
#     AA['b'].loc[k] = [np.random.random((5,3))]


In [ ]:
5 in [5, 10, 15]

In [ ]:
AA['b'].to_numpy().mean(axis=0)

In [ ]:
data.mean(axis=0)

In [ ]:
AA = pd.DataFrame({'a':[15 ,18, 11, 7], 'b':[21, 19, 20, 2]})

In [ ]:
AA.head()

In [ ]:
AA.agg([lambda x: np.linalg.norm(x, axis=0)])

In [ ]:
AA.iloc[1:].agg('idxmax')

In [ ]:
xx = np.array([3, 4])
np.argmax(xx)